In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 1. 設定 (既存環境継承)
SUBSPACE_PATH = "common_subspace.pt"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# -----------------------------------------------------------
# 🧠 Internal Divergence Curiosity (IDC) モデル定義
# -----------------------------------------------------------
class IDCRewardModel:
    def __init__(self, basis, mean, device, memory_capacity=2048):
        """
        Args:
            basis: PCA基底 (共通成分)
            mean: PCA平均
            memory_capacity: 記憶する過去のトークン/ベクトルの最大数
        """
        self.device = device
        self.basis = basis.to(device)
        self.mean = mean.to(device)
        
        # エピソード記憶 (FIFOバッファ)
        self.memory_capacity = memory_capacity
        self.memory_buffer = torch.empty(0, basis.shape[1], device=device, dtype=basis.dtype)
        
    def get_semantic_vector(self, h):
        """
        隠れ状態 h から「意味論ベクトル(残差)」を抽出する
        Common Subspace (平凡な成分) を除去したものを使用
        """
        h = h.to(self.basis.dtype)
        h_centered = h - self.mean
        
        # 共通成分の算出と除去
        # h_common = (h_centered @ basis.T) @ basis
        z_common = h_centered @ self.basis.T
        h_common = z_common @ self.basis
        
        # 残差ベクトル = 意味論ベクトル (Semantic Vector)
        h_semantic = h_centered - h_common
        return h_semantic

    def compute_reward_and_update(self, h_batch):
        """
        バッチ内の各隠れ状態について、記憶との距離(報酬)を計算し、記憶を更新する
        Args:
            h_batch: (Batch, Dim) or (Seq, Dim) の隠れ状態
        Returns:
            rewards: (Batch,) 各ステップの拡散的好奇心報酬
        """
        # 1. 意味論ベクトルの抽出
        current_vectors = self.get_semantic_vector(h_batch)
        
        # 2. 正規化 (コサイン類似度計算のため)
        current_vectors_norm = F.normalize(current_vectors, p=2, dim=1)
        
        rewards = []
        
        # メモリが空の場合の処理
        if self.memory_buffer.shape[0] == 0:
            # 最初は比較対象がないので報酬はベースライン(例: 0.5)や最大値を与える
            # ここでは「過去がない=最大の驚き」として 1.0 を与える
            rewards = torch.ones(current_vectors.shape[0], device=self.device)
        else:
            # 3. 記憶との類似度計算 (Matrix Multiplication)
            # Memoryは既に正規化されていると仮定して計算コスト削減も可能だが、
            # ここでは安全のため都度正規化する実装とする
            memory_norm = F.normalize(self.memory_buffer, p=2, dim=1)
            
            # Cosine Similarity: [Batch, Memory_Size]
            sim_matrix = torch.mm(current_vectors_norm, memory_norm.T)
            
            # 4. 「最も似ている過去」を探す (Max Similarity)
            # 既視感 = Max Sim.  拡散的好奇心 = 1 - 既視感
            max_sim, _ = torch.max(sim_matrix, dim=1)
            
            # 報酬計算 (距離)
            # Simは -1~1 なので、 1 - Sim は 0~2 の範囲になる
            rewards = 1.0 - max_sim
            
        # 5. 記憶の更新 (現在のベクトルを記憶に追加)
        # 次のステップのために、今の「新しい体験」を記憶する
        self.memory_buffer = torch.cat([self.memory_buffer, current_vectors.detach()], dim=0)
        
        # 容量オーバーしたら古い記憶を捨てる (FIFO)
        if self.memory_buffer.shape[0] > self.memory_capacity:
            self.memory_buffer = self.memory_buffer[-self.memory_capacity:]
            
        return rewards

    def clear_memory(self):
        """エピソードごとのリセット用"""
        self.memory_buffer = torch.empty(0, self.basis.shape[1], device=self.device, dtype=self.basis.dtype)


# -----------------------------------------------------------
# 🧪 検証コード: 平凡な繰り返し vs 創造的な展開
# -----------------------------------------------------------

# 1. モデルとサブスペースのロード (前のセルでロード済みなら省略可)
try:
    # 既存の basis, mean があればそれを使う
    idc_model = IDCRewardModel(basis, mean, DEVICE, memory_capacity=512)
    print("IDC Model Initialized with existing Subspace.")
except NameError:
    print("Loading Subspace...")
    data = torch.load(SUBSPACE_PATH, map_location=DEVICE, weights_only=False)
    basis = data["basis"].to(dtype=torch.float16)
    mean = data["mean"].to(dtype=torch.float16)
    idc_model = IDCRewardModel(basis, mean, DEVICE, memory_capacity=512)

# 2. テストプロンプト
# Prompt A: 同じトピックを繰り返す (報酬が下がるはず)
# Prompt B: 常に新しい展開をする (報酬が維持されるはず)
prompts = [
    {
        "type": "Repetitive (Dejà Vu)",
        "text": "The cat sat on the mat. The cat sat on the mat. The cat sat on the mat. The cat sat on the mat."
    },
    {
        "type": "Diverse (Diffusive)",
        "text": "The cat sat on the mat. Suddenly, a spaceship landed in the garden. Aliens came out and started dancing tango."
    }
]

# 3. 実行と可視化
plt.figure(figsize=(12, 6))

for p_data in prompts:
    label = p_data["type"]
    text = p_data["text"]
    
    # 記憶をリセット (エピソード開始)
    idc_model.clear_memory()
    
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    
    # 隠れ状態シーケンス [Seq_Len, Dim]
    # バッチサイズ1前提
    h_seq = outputs.hidden_states[-1].squeeze(0)
    
    # 逐次的に報酬を計算 (時系列変化を見るため)
    step_rewards = []
    
    # 最初のトークンから順番に処理
    # (実際はまとめて計算できますが、記憶の更新過程をシミュレートするためループ)
    for t in range(h_seq.shape[0]):
        h_t = h_seq[t].unsqueeze(0) # [1, Dim]
        r_t = idc_model.compute_reward_and_update(h_t)
        step_rewards.append(r_t.item())
    
    # プロット
    tokens = [tokenizer.decode(t) for t in inputs.input_ids[0]]
    plt.plot(step_rewards, marker='o', label=label)
    
    # 平均スコア表示
    avg_score = sum(step_rewards) / len(step_rewards)
    print(f"Type: {label:20s} | Avg IDC Reward: {avg_score:.4f}")

plt.title("IDC Reward Transition: Repetitive vs Diverse")
plt.xlabel("Token Step")
plt.ylabel("IDC Reward (1 - Max Similarity)")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()